In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Directories
dir_word = 'img-word'
dir_noise = 'img-noise'

# List of images
images_word = sorted([os.path.join(dir_word, file) for file in os.listdir(dir_word) if file.endswith(('png', 'jpg', 'jpeg'))])
images_noise = sorted([os.path.join(dir_noise, file) for file in os.listdir(dir_noise) if file.endswith(('png', 'jpg', 'jpeg'))])

# Plot images
fig, axs = plt.subplots(len(images_word), 2, figsize=(10, 5 * len(images_word)))

for i, (img_word_path, img_noise_path) in enumerate(zip(images_word, images_noise)):
    img_word = mpimg.imread(img_word_path)
    img_noise = mpimg.imread(img_noise_path)
    
    # Display word image
    axs[i, 0].imshow(img_word)
    axs[i, 0].set_title(f'Image Word {i+1}')
    axs[i, 0].axis('off')
    
    # Display noise image
    axs[i, 1].imshow(img_noise)
    axs[i, 1].set_title(f'Image Noise {i+1}')
    axs[i, 1].axis('off')

plt.tight_layout()
# plt.show() #hide to show

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import random

In [ ]:

# Helper function to add noise to a pattern
def add_noise(pattern, noise_level):
    noisy_pattern = pattern.copy()
    num_noisy = int(noise_level * pattern.size)
    indices = np.random.choice(pattern.size, num_noisy, replace=False)
    noisy_pattern.flat[indices] *= -1
    return noisy_pattern

# Convert image to binary pattern
def convert_image_to_binary(path):
    image = Image.open(path).convert('L')
    image = image.resize((10, 10))  # Resize for simplicity
    binary_pattern = np.array(image)
    binary_pattern = (binary_pattern > 128).astype(int) * 2 - 1
    return binary_pattern

# Save binary pattern as image
def save_binary_image(pattern, path):
    pattern = (pattern + 1) * 127.5  # Scale back to 0-255
    image = Image.fromarray(pattern.astype(np.uint8))
    image.save(path)




In [ ]:

class HopfieldNetwork:
    def __init__(self, num_neurons):
        self.num_neurons = num_neurons
        self.weights = np.zeros((num_neurons, num_neurons))

    def train(self, patterns):
        '''Train the network using Hebbian learning.'''
        for pattern in patterns:
            pattern = np.array(pattern).flatten()
            self.weights += np.outer(pattern, pattern)
        np.fill_diagonal(self.weights, 0)
        self.weights /= len(patterns)

    def recall(self, pattern, steps=10):
        '''Recall a pattern from the network.'''
        pattern = np.array(pattern).flatten()
        for _ in range(steps):
            for i in range(self.num_neurons):
                raw_value = np.dot(self.weights[i], pattern)
                pattern[i] = 1 if raw_value >= 0 else -1
        return pattern.reshape(int(np.sqrt(self.num_neurons)), -1)

    def energy(self, pattern):
        '''Calculate the energy of a given pattern.'''
        pattern = np.array(pattern).flatten()
        return -0.5 * pattern.T @ self.weights @ pattern


In [ ]:

# Load and convert image to binary
original_pattern = convert_image_to_binary('/content/5.jpg')

# Create Hopfield network
net = HopfieldNetwork(num_neurons=original_pattern.size)
net.train([original_pattern])

# Add noise to the image
noisy_pattern = add_noise(original_pattern, noise_level=0.3)

# Recall the image
recalled_pattern = net.recall(noisy_pattern)

# Save results
save_binary_image(original_pattern, '/content/original.jpg')
save_binary_image(noisy_pattern, '/content/noisy.jpg')
save_binary_image(recalled_pattern, '/content/recalled.jpg')

# Plot results
fig, axs = plt.subplots(1, 3, figsize=(12, 4))
axs[0].imshow(original_pattern, cmap='gray')
axs[0].set_title('Original Pattern')
axs[1].imshow(noisy_pattern, cmap='gray')
axs[1].set_title('Noisy Pattern')
axs[2].imshow(recalled_pattern, cmap='gray')
axs[2].set_title('Recalled Pattern')
plt.show()

In [ ]:
# Helper function to create Point Missing pattern
def add_point_missing(pattern, missing_ratio):
    '''Add point missing noise to the pattern.'''
    missing_pattern = pattern.copy()
    num_missing = int(missing_ratio * pattern.size)
    indices = np.random.choice(pattern.size, num_missing, replace=False)
    missing_pattern.flat[indices] = 0
    return missing_pattern

In [ ]:
original_pattern = convert_image_to_binary('/content/5.jpg')

# Create Hopfield network
net = HopfieldNetwork(num_neurons=original_pattern.size)
net.train([original_pattern])

# Add Point Missing to the image
point_missing_pattern = add_point_missing(original_pattern, missing_ratio=0.3)

# Recall the image
recalled_pattern_pm = net.recall(point_missing_pattern)

# Save results
save_binary_image(original_pattern, '/content/original.jpg')
save_binary_image(point_missing_pattern, '/content/point_missing.jpg')
save_binary_image(recalled_pattern_pm, '/content/recalled_point_missing.jpg')

# Plot results
fig, axs = plt.subplots(1, 3, figsize=(12, 4))
axs[0].imshow(original_pattern, cmap='gray')
axs[0].set_title('Original Pattern')
axs[1].imshow(point_missing_pattern, cmap='gray')
axs[1].set_title('Point Missing Pattern')
axs[2].imshow(recalled_pattern_pm, cmap='gray')
axs[2].set_title('Recalled Pattern (Point Missing)')
plt.show()
